
# Análisis de textos

El mundo del análisis de textos es totalmente diferente a lo visto hasta ahora, con sus propios problemas y bibliotecas especializadas. Vamos a ver solo algunas de sus características.

Empecemos asegurándonos de que las bibliotecas necesarias están instaladas


In [ ]:
modules = ["wikipedia",  "bs4", "spacy", "nltk","sentiment_analysis_spanish","flair"]


import sys
import os.path
from subprocess import check_call
import importlib
import os

def instala(modules):
  print("Instalando módulos")
  for m in modules:
      # para el import quitamos [...] y ==...
      p = m.find("[")
      mi = m if p==-1 else m[:p]
      p = mi.find("==")
      mi = mi if p==-1 else mi[:p]

      torch_loader = importlib.util.find_spec(mi)
      if torch_loader is not None:
          print(m," encontrado")
      else:
          print(m," No encontrado, instalando...",end="")
          try:
            r = check_call([sys.executable, "-m", "pip", "install", "--user", m])
            print("¡hecho!")
          except:
            print("¡Problema al instalar ",m,"! ¿seguro que el módulo existe?",sep="")

  print("¡Terminado!")

instala(modules)

## Tokens

La primera idea es obtener las piezas sintácticas básicas, palabras, símbolos de puntuación, etc., pero esto no es tan trivial como parece. Empezamos usando la función estándar `split`

In [ ]:
s = "Desde lo alto se divisa la ciudad y toda la campiña"
l = s.split(" ")
l

In [ ]:
s = "Se remueve, levanta una tenue polvareda, avanza."
l = s.split(" ")
l

Bibliotecas como spaCy permiten obtener mejor los tokens

In [ ]:
import spacy
nlp = spacy.blank('es') # modelo
s = "Se remueve, levanta una tenue polvareda, avanza."
doc =  nlp(s)
l = [token.text for token in doc]
l

Podemos "limpiar" el texto quedando solo con un tipo de palabras

In [ ]:
l =  [token.text for token in doc if  token.is_alpha]
l

Un caso particular de *token* son las stop words, palabras sin semántica que pueden desvirtuar nuestros análisis

In [ ]:
l =  [token.text for token in doc if  token.is_alpha and not token.is_stop]
l

### Contando oraciones y palabras

Igual que con las palabras podemos intentarlo desde Python estándar:

In [ ]:
s = "he comprado patatas, naranjas, manzanas..."
punto = s.find(".") # posición del primer punto
if punto!=-1: # -1 si no hay puntos
  print("La primera oración: ", s[:punto])
  print("El resto del texto: ", s[punto+1:])

Spacy tiene "modelos pre-entrenados". Descargamos uno de ellos

In [ ]:
!python -m spacy download es_core_news_sm

In [ ]:
import es_core_news_sm
nlp = es_core_news_sm.load()
s = "he comprado patatas, naranjas, manzanas... todo en \
     http://verdurasfrescas.com. ¡Y ha llegado muy rápido!"
doc = nlp(s)
for i,o in enumerate(doc.sents):
    print(i+1,o)


In [ ]:
from bs4 import BeautifulSoup
import requests
pagina = "https://www.gutenberg.org/cache/epub/8870/pg8870.html"
page = requests.get(pagina)
soup = BeautifulSoup(page.text, 'html.parser')  # le pasamos el texto en HTML para que lo analice
soup.text

In [ ]:
doc =  nlp(soup.text)
print("El libro incluye ", len(list(doc.sents)), "oraciones")

Si lo que queremos es contar palabras podemos utilizar `Counter` que incluye un diccionario con cada letra de la palabra y su frecuencia.

In [ ]:
from collections import Counter
palabras =  [token.text for token in doc if  token.is_alpha and not token.is_stop]
c = Counter(palabras)

In [ ]:
c

In [ ]:
c.most_common(20)

Podemos añadir "the" como stopword

In [ ]:
nlp = spacy.blank("es")
nlp.Defaults.stop_words.add("the")
doc =  nlp(soup.text)
palabras =  [token.text for token in doc if  token.is_alpha and not token.is_stop]
c = Counter(palabras)
c.most_common(20)

### Lemas y raíces

El resultado anterior se ve afectado por el hecho de que la misma palabra puede aparecer en singular, plural, verbos conjugados de distinta forma, etc.

In [ ]:
import es_core_news_sm
nlp = es_core_news_sm.load()
doc =  nlp(soup.text)
palabras = [p.lemma_.upper() for p in doc
            if p.is_alpha and not p.is_stop]
c = Counter(palabras)
c.most_common(20)

La **extracción de raíces**, conocida en la literatura en inglés como *stemming*, que convierte palabras a su raíz eliminando sufijos de persona, género, etc. Es una alternativa a la lematización que ha sido tradicionalmente utilizada en aplicaciones de inteligencia artificial, por ejemplo en *aprendizaje automático*, pero que no es tan útil para nuestros propósitos al no obtenerse palabras completas como resultados

In [ ]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')
palabras = [stemmer.stem(p.text).upper() for p in doc
              if p.is_alpha and not p.is_stop]
frecuencia = Counter(palabras)
palabras_comunes = frecuencia.most_common(5)

palabras_comunes

In [ ]:
from nltk.stem import SnowballStemmer
import es_core_news_sm
nlp = es_core_news_sm.load()
doc = nlp(soup.text)

stemmer = SnowballStemmer('spanish')

palabras = [p.text.upper() for p in doc
             if p.is_alpha and not p.is_stop]
print("Palabras no vacías en el texto ", len(set(palabras)))

palabras = [p.lemma_.upper() for p in doc
            if p.is_alpha and not p.is_stop]
print("Con lemas ", len(set(palabras)))
palabras = [stemmer.stem(p.text).upper() for p in doc
              if p.is_alpha and not p.is_stop]
print("Con raíces ", len(set(palabras)))

## Análisis de sentimiento

In [ ]:
# origen https://www.amazon.com/-/es/product-reviews/B07ZPKF8RG/ref=cm_cr_unknown?ie=UTF8&filterByStar=two_star&reviewerType=all_reviews&pageNumber=1#reviews-filter-bar
textos = ['Broken. It was supposed to be a B-Day gift for my mom and now she doesn’t get a gift on her Bday',
          'Touchscreen became almost completely unresponsive over 20% of surface area within days',
          "A little over 90 days, hardware failure, NO Solution, DON'T BUY",
          "Wanted to love it, but it had too many problems upon arrival",
          "Phone is great but the battery health is 76%",
          "Happy customer! It came with protective glass installed.",
          "Great phone for the price!",
          "I love this phone",
          "Definitely Recommend!"]

In [ ]:
from flair.models import TextClassifier
from flair.data import Sentence
classifier = TextClassifier.load('en-sentiment')

for s in textos:
    text = Sentence(s)
    classifier.predict(text)
    value = text.labels[0].to_dict()
    print(s)
    print(value)
    print("="*30)

Aplicándoselo a una columna de Pandas

In [ ]:
import pandas as pd
df = pd.read_csv("https://github.com/RafaelCaballero/tdm/blob/master/datos/IMDB10K.zip?raw=true",compression='zip')
total = 250
df = df.sample(250)
df

In [ ]:
from tqdm import tqdm
flairOp = []
for s in tqdm(df.review):
    text = Sentence(s)
    classifier.predict(text)
    value = text.labels[0].to_dict()["value"]
    flairOp.append(+1 if value=="POSITIVE" else 0)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

y_real = [+1 if v=="positive" else 0 for v in df.sentiment]

cm = confusion_matrix(y_real, flairOp)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

## NER (Named Entity Recognition)

Reconocimiento de sustantivos que denotan nombres, organizaciones, etc. Importante para saber de qué se está hablando en textos complejos

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

# load tagger
tagger = SequenceTagger.load("flair/ner-spanish-large")

In [ ]:


# make example sentence
sentence = Sentence("Belén fue a Belén el mismo día que Santiago llegó a Santiago de Compostela")

# predict NER tags
tagger.predict(sentence)

# print sentence
print(sentence)

# print predicted NER spans
print('The following NER tags are found:')
# iterate over entities and print
for entity in sentence.get_spans('ner'):
    print(entity)